In [53]:
import matplotlib.pyplot as plt
import numpy as np
import random
import copy
from IPython.display import clear_output
import matplotlib.animation as animation

In [54]:
class Lenia:
        def __init__(self, n, m, kernel_outer_radius, kernel_inner_radius, iters):

            self.iters = iters
            self.n = n
            self.m = m
            self.matrix = np.zeros((n, m))
            self.kernel = self.create_kernel(kernel_outer_radius, kernel_inner_radius, smoothness=3)
            self.shape_kernel = len(self.kernel), len(self.kernel[0])
            
            # -- animation
            self.count_loop = 0
            self.animation = None
            
    
            
        def load_file(self, file):
            lista = []
            with open(file, 'r') as file:
                for line in file:
                    lista.append(list(map(lambda e: float(e), line.replace('\n', '').split())))
            for i in range(len(lista)):
                self.matrix[int(lista[i][1])][int(lista[i][0])] = 1
                
        def load_points(self, points_x: list, points_y: list):
            if len(points_x) != len(points_y):
                raise Exception('Lists are not eaqual!')
            for i in range(len(points_x)):
                self.matrix[points_y[i]][points_x[i]] = 1
                
                
        # -------------------------------------
        def create_kernel(self, outer_radius, inner_radius, smoothness):
            size = 2 * outer_radius + 1
            kernel = np.zeros((size, size))
            center = outer_radius
        
            for i in range(size):
                for j in range(size):
                    distance = np.sqrt((i - center) ** 2 + (j - center) ** 2)
                    if distance <= inner_radius:
                        kernel[i, j] = 0  # Inside the inner circle, values are 0
                    elif distance <= outer_radius:
                        # Create a smooth transition from 0 to 1 using a smoothness factor
                        kernel[i, j] = (distance - inner_radius) / (outer_radius - inner_radius) ** smoothness
        
            return kernel
        
        def calc_U(self, matrix, i_c, j_c):
            u = 0
            count_k = 0
    
            for i_k in range(self.shape_kernel[0]):
                for j_k in range(self.shape_kernel[1]):
                    i_matrix_index = i_c - int(self.shape_kernel[0]/2) + i_k
                    j_matrix_index = j_c - int(self.shape_kernel[1]/2) + j_k
                    if  (0 <= i_matrix_index < len(matrix)) and (0 <= j_matrix_index < len(matrix[0])):
                        u += matrix[i_matrix_index][j_matrix_index] * self.kernel[i_k][j_k]
                        count_k += self.kernel[i_k][j_k]
                    
            u = u / count_k
            return u
        
        
        def growth_func(self, u):
            sigma = 0.01
            mu = 0.1
            l = abs(u - mu)
            k = 2 * (sigma ** 2)
            return 2 * np.exp(-(l ** 2) / k) - 1
        
        def calc_c_t(self, matrix, i, j):
            u = self.calc_U(matrix, i, j)
            a = self.growth_func(u)
            at = 0.1
            return np.clip((matrix[i][j] + at*a), 0, 1)
        
        def core_funct(self):
            fig= plt.figure(figsize=(10, 8))
            im = plt.imshow(self.matrix, cmap='magma', animated=True)
            
            def animation_loop(frame):
                matrix_tmp = np.zeros((self.n,self.m))
                for i in range(self.matrix.shape[0]):
                    for j in range(self.matrix.shape[1]):
                        matrix_tmp[i][j] = self.calc_c_t(self.matrix, i, j)
                        
                self.matrix = copy.deepcopy(matrix_tmp)
                im.set_array(self.matrix)
                plt.title(f'Generation: {self.count_loop}')
                self.count_loop += 1
                return im,
            
            self.animation = animation.FuncAnimation(fig, func=animation_loop, frames=self.iters, interval=10, cache_frame_data=False)
            self.animation.save('lenia.gif')
              
                        
        
            
            

In [ ]:
lenia = Lenia(n=200,m=200, kernel_outer_radius=9, kernel_inner_radius=4, iters=300)
lenia.load_points(points_x=[random.randint(100,120) for _ in range(100)], points_y=[random.randint(100,120) for _ in range(100)])
lenia.core_funct()

MovieWriter ffmpeg unavailable; using Pillow instead.
